Your model will be evaluated using two metrics: profit @ top-20, and AUC. The reasons for this is to be in line with a more realistic setting. E.g. one can image data scientists in a team arguing to use AUC and optimize for that. However, as seen in the course, for this scenario, we also imagine management arguing that there is not enough budget (in terms of time and money) to contact a lot of people (or hand out a lot of promotions). Hence, they have come up with the following: based on the top-k would-be churners as predicted by your model, sum some proxy of "retained profitability" in case the customer was indeed a churner, or zero otherwise

As a proxy of profitability, the feature average cost min was deemed to be a good value. Based on the size of the test set, k=20 was deemed to be a good choice. Hence, management cares about optimizing this metric
Note that only about half of the test set is used for the "public" leaderboard. That means that the score you will see on the leaderboard is done using this part of the test only (you don't know which half). Later on through the semester, submissions are frozen and the resuls on the "hidden" part will be revealed

Also, whilst you can definitely try, the goal is not to "win", but to help you reflect on your model's results, see how others are doing, etc.

Objectives:

Some groups prefer to write their final report using Jupyter Notebook, which is fine too, as long as it is readable top-to-bottom

You can use any predictive technique/approach you want, though focus on the whole process: general setup, critical thinking, and the ability to get and validate an outcome

You're free to use unsupervised technique for your data exploration part, too. When you decide to build a black box model, including some interpretability techniques to explain it is a plus

Any other assumptions or insights are thoughts can be included as well: the idea is to take what we've seen in class, get your hands dirty and try out what we've seen

Perform a critical review of the evaluation metric chosen by management. How in line is it with AUC? What would you have picked instead? Were there particular issues with this chosen metric, in your view?

In [ ]:
# !pip install shap

In [ ]:
import pandas as pd
import os
import plotly.graph_objects as go
import numpy as np
import shap

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.svm import SVC

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [ ]:
# Initialising
TRAIN_SET_FRAC = 0.8
SEED = 42
TARGET_VAR = "target"
DROP_VARS = ['Connect_Date', 'id'] # TBC
KFOLD = 5

**Loading Data**

In [ ]:
# GitHib urls to fetch data from
url_train = 'https://raw.githubusercontent.com/hello-bob/AA_P1/main/data/train.csv'
url_test = 'https://raw.githubusercontent.com/hello-bob/AA_P1/main/data/test.csv'

# Read train and test data
train_data = pd.read_csv(url_train, sep = ',', skipinitialspace = True, engine = 'python')
train_data = train_data.drop(columns=DROP_VARS)
test_data  = pd.read_csv(url_test, sep = ',', skipinitialspace = True, engine = 'python')

**Data exploration**

In [ ]:
train_data.head()

In [ ]:
# Check data types
train_data.info()
test_data.info()

In [ ]:
# Basic descriptives
train_data.describe(include='all')

**Data Cleaning**

In [ ]:
# Impute missing data before modelling: Can quantitate and put it on the report since 4/5k samples
# Apply on the test set. Train set is ok.

train_data.isnull().any().sort_values(ascending=False) # Columns with missing values: Dropped_calls_ratio, Usage_Band, call_cost_per_min.
train_data[train_data.isnull().any(axis=1)] # 4 cases, 2 churners

imputer_compiled = ColumnTransformer(
    [("numeric_imputer", SimpleImputer(strategy="median",), ["Dropped_calls_ratio", "call_cost_per_min"]),
     ("cat_imputer", SimpleImputer(strategy="most_frequent"), ["Usage_Band"])]
)

# Imput median for numeric variables first. Because "most_frequent" strategy will impute for both numeric and categorical data
train_data[["Dropped_calls_ratio", "call_cost_per_min", "Usage_Band"]] = imputer_compiled.fit_transform(train_data)
test_data[["Dropped_calls_ratio", "call_cost_per_min", "Usage_Band"]] = imputer_compiled.transform(test_data)

# Correcting dtype
train_data[["Dropped_calls_ratio", "call_cost_per_min"]] = train_data[["Dropped_calls_ratio", "call_cost_per_min"]].astype(float)
test_data[["Dropped_calls_ratio", "call_cost_per_min"]] = test_data[["Dropped_calls_ratio", "call_cost_per_min"]].astype(float)


**Exploratory**

In [ ]:
# [For report] Pie chart about class inbalance (train set) + Percentage churn in categorical variable


In [ ]:
# [For report] correlation plot
corr = train_data.corr(numeric_only=True)

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corr.columns,
        y = corr.index,
        z = np.array(corr),
        text=corr.values,
        texttemplate='%{text:.2f}'
    )
)
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)
fig.show()

In [ ]:
# Print correlations which has >0.7
"""
All_calls_mins highly correlated to National minutes. It's a sum of National minutes + International mins maybe. This may indicate that
majority of calls are within nation. This corresponds to how Nat_call_cost_Sum is highly correlated to actual call cost. Not sure what's the
diff betwen actuall call cost and total call cost. Nat_call_cost_Sum could be an adjustment of actual call cost, correlation 0.999.

Total_call_cost most strongly correlated with International_mins_Sum and Total_Cost. This may suggest that costs are largely driven by 
international calls

Not sure what total calls indicate, maybe it's cost from other telco-related services e.g. broadband, cable tv etc.

Peak_mins_Sum highly correlated with all_calls_mins and national mins. This may make sense since the more minutes of calling, the higher
likelihood of calling during the peak period?

"""
corr_long = train_data.corr(numeric_only=True).melt(ignore_index=False).reset_index(drop=False)
corr_long = corr_long[(corr_long['index'] != corr_long['variable']) & (abs(corr_long['value']) >0.7)]
corr_long.sort_values(by=['index', 'variable'], ascending=True)

In [ ]:
(round((train_data['International_mins_Sum'] + train_data['National mins']),2) == round(train_data['All_calls_mins'], 2)).sum() #almost all
train_data[['All_calls_mins', 'National mins']]

In [ ]:
train_data.select_dtypes('object').columns.to_list()

In [ ]:
# [For report] Correlation between categorical variables, using cramer's V
# Generally, 
from scipy.stats.contingency import association

cat_corr_cols = train_data.select_dtypes('object').columns.to_list()
while len(cat_corr_cols) > 0:
    col = cat_corr_cols.pop(0)
    print(f'Correlation for {col}')
    for other_col in cat_corr_cols:
        contingency_tbl = pd.crosstab(train_data[col], train_data[other_col])
        cramer_V = association(contingency_tbl, method="cramer")
        print(f'Association with {other_col}: {cramer_V}')
    print('\n')
    


In [ ]:
pd.crosstab(train_data['tariff'], train_data['Usage_Band'])
pd.crosstab(train_data['Gender'], train_data['Handset'])
pd.crosstab(train_data['high Dropped calls'], train_data['Handset'])

In [ ]:
# On the metric by management
train_data.sort_values(by='average cost min', ascending=False).head(20)

In [ ]:
# Identifying outliers via isolation forest
from sklearn.ensemble import IsolationForest

outlier_df = (train_data.select_dtypes(include='number')
              .drop(columns=TARGET_VAR)
              .dropna()
              .copy())

iso_forest = IsolationForest(random_state=SEED, n_jobs=-1, contamination=0.05).fit(outlier_df)

In [ ]:
outlier_df['outlier_score'] = iso_forest.decision_function(outlier_df) # more negative indicates higher outlier-ness

In [ ]:
# https://stats.stackexchange.com/questions/404017/how-to-get-top-features-that-contribute-to-anomalies-in-isolation-forest
"""
Outstanding variables contributing to outlier (to the left of 0 on x axis) are AveOffPeak, average cost min, AveWeekend, AveNational and 
Dropped_calls_ratio. These tend to indicate the higher the values, the more of an outlier they are.
"""

# Create shap values and plot them
shap_values = shap.TreeExplainer(iso_forest).shap_values(outlier_df)
shap.summary_plot(shap_values, outlier_df, plot_type='violin')

In [ ]:
# Average shap per variable Top few: Weekend_calls_sum, nat_call_cost_sum, dropped_calls, average cost min
# a global measure of feature importance (https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html)
# These values seem low to the values I saw online. 
explainer = shap.Explainer(iso_forest, outlier_df)
shap_values = explainer(outlier_df)
shap.plots.bar(shap_values)

In [ ]:
# 3 of the top 10 outliers are churners. No discernible pattern between churn and non-churn. Propose to keep all and compare 
# between models which are robust against outliers, and those not.
outlier_df[TARGET_VAR] = train_data[TARGET_VAR].copy()
outlier_df.sort_values(by='outlier_score', ascending=True).head(10).sort_values(by='target')
outlier_df.groupby('target').mean()

**Modelling**

In [ ]:
X = train_data.drop(columns=TARGET_VAR)
y = train_data[TARGET_VAR] 

NUM_VARS = train_data.select_dtypes(include='number').drop(columns=TARGET_VAR).columns
CAT_VARS = train_data.select_dtypes(include='object').columns

In [ ]:
print(NUM_VARS)
print(CAT_VARS)

In [ ]:
# Define preprocessors for numerical and categorical features
numerical_preprocessor = Pipeline([
    ("scaler", StandardScaler())
])

categorical_preprocessor = Pipeline([
    ("onehot", OneHotEncoder(drop="if_binary"))
])

In [ ]:
# Combine preprocessors and model
model = Pipeline([
    ("preprocessor", ColumnTransformer([
        ("numerical", numerical_preprocessor, NUM_VARS),
        ("categorical", categorical_preprocessor, CAT_VARS)
    ])),
    ("model", SVC(probability=False, random_state=SEED, max_iter=10000))
])
model

In [ ]:
# For SVM
parameters = {'model__kernel':['linear', 'rbf', 'sigmoid', 'poly'], 
              'model__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000], 
              'model__gamma':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
             'model__class_weight':[None, 'balanced'],
             'model__degree':[3,4,5]} # rmb to add the double underscores to allow gridsearch to fit on pipelines
svc_gs_est = GridSearchCV(estimator=model, param_grid=parameters,cv=KFOLD,
                      scoring="roc_auc",n_jobs=-1, refit=False, verbose=10)
svc_gs_est.fit(X, y)

In [ ]:
svc_gs_est

In [ ]:
svc_gs_results = pd.DataFrame(data=svc_gs_est.cv_results_)
svc_gs_results.sort_values(by='rank_test_score', ascending = True).to_csv('output/svc_gridsearchcv.csv')
svc_gs_results.sort_values(by='rank_test_score', ascending = True)

In [ ]:
# Best params
svc_gs_est.best_params_

In [ ]:
# Retrain best model: Set up the params accordingly
numerical_preprocessor = Pipeline([
    ("scaler", StandardScaler())
])

categorical_preprocessor = Pipeline([
    ("onehot", OneHotEncoder(drop="if_binary"))
])

best_svc_model = Pipeline([
    ("preprocessor", ColumnTransformer([
        ("numerical", numerical_preprocessor, NUM_VARS),
        ("categorical", categorical_preprocessor, CAT_VARS)
    ])),
    ("model", SVC(probability=False, random_state=SEED, C=100, class_weight='balanced', kernel="rbf"))
])


best_svc_model.fit(X, y)

In [ ]:
# For submission
test_data_sub = pd.DataFrame(data={'ID':test_data['id'], 
                                   'PRED':best_model.predict(test_data)})
test_data_sub

**XGBoost**

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

In [ ]:
# Basic preprocessing
numeric_transformer = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy="median"))
    ]
)

categorical_transformer = Pipeline(
    steps = [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, NUM_VARS),
        ("cat", categorical_transformer, CAT_VARS)
    ]
)

X_preprocessed = preprocessor.fit_transform(X)
# Alternatively split train-test before, do preprocessing on training data (fit_transform) then transform test data


In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X_preprocessed, y, test_size=0.2, stratify=y, random_state=420)

Grid search

In [ ]:
# Calcualte class ratio. Will be used to assess class imbalance during training
ratio = float(y_train.value_counts()[0]) / y_train.value_counts()[1]

In [ ]:
# First do a random search over a large hyperparameter space, trying 1000 random models 
gbm_param_grid_large = {  'n_estimators': np.arange(5, 101, 1)
                        , 'max_depth': range(2, 13)
                        , 'learning_rate': np.arange(0.001, 5, 0.01)
                        , 'subsample': [0.2, 0.4, 0.6, 0.8, 1]
                        , 'colsample_bytree': [0.2, 0.5, 0.8, 1]
                        , 'reg_lambda': [0, 1, 5, 10, 100]
                        }

gbm = xgb.XGBClassifier(random_state=420, scale_pos_weight=ratio)
randomized_auc = RandomizedSearchCV(  estimator=gbm
                                    , param_distributions=gbm_param_grid_large
                                    , n_iter=1000
                                    , scoring='roc_auc'
                                    , cv=5
                                    , n_jobs=-1
                                    , verbose=1
                                    , random_state=420)

# See how the cross-validation performed (on the "training" data) and the best tuned hyperparameter values
randomized_auc.fit(X_train, y_train)
print("Best parameters found: ",randomized_auc.best_params_)
print("Lowest AUC found: ", randomized_auc.best_score_)

In [ ]:
# Now do another random search over a smaller hyperparameter space around the preivously found "best" values
gbm_param_grid_medium = {  'n_estimators': np.arange(50, 120, 1)
                         , 'max_depth': range(6, 15)
                         , 'learning_rate': np.arange(0.001, 5, 0.01)
                         , 'subsample': [0.6, 0.8, 1]
                         , 'colsample_bytree': [0.8, 1]
                         , 'reg_lambda': [0, 1, 2, 5]
                         }

gbm = xgb.XGBClassifier(random_state=420, scale_pos_weight=ratio)
randomized_auc = RandomizedSearchCV(  estimator=gbm
                                    , param_distributions=gbm_param_grid_medium
                                    , n_iter=1000
                                    , scoring='roc_auc'
                                    , cv=5
                                    , n_jobs=-1
                                    , verbose=1
                                    , random_state=420)

# See how the cross-validation performed (on the "training" data) and the best tuned hyperparameter values
randomized_auc.fit(X_train, y_train)
print("Best parameters found: ",randomized_auc.best_params_)
print("Lowest AUC found: ", randomized_auc.best_score_)

In [ ]:
# We got quite different reults, but different hyperparameter combinations can give similar results.
# Finally a grid-search that is not random around the previous "best" values.
gbm_param_grid = {  'n_estimators': [70, 80, 100, 120]
                  , 'max_depth': [8, 10, 12]
                  , 'learning_rate': [0.05, 0.1, 0.15, 0.2]
                  , 'subsample': [0.8, 1]
                  , 'colsample_bytree': [0.8, 1]
                  , 'reg_lambda': [0, 1, 2]
                  }

gbm = xgb.XGBClassifier(random_state=420, scale_pos_weight=ratio)
grid_auc = GridSearchCV(  estimator=gbm
                        , param_grid=gbm_param_grid
                        , scoring='roc_auc'
                        , cv=5
                        , n_jobs=-1
                        , verbose=1
                        )

# See how the cross-validation performed (on the "training" data) and the best tuned hyperparameter values
grid_auc.fit(X_train, y_train)
print("Best parameters found: ", grid_auc.best_params_)
print("Lowest AUC found: ", grid_auc.best_score_)

In [ ]:
# Save an output based on the previous (last) grid search
pd.DataFrame(grid_auc.cv_results_).sort_values(by='rank_test_score', ascending = True).to_csv('output/xgb_gridsearch.csv')

In [ ]:
# Get an idea about how the model performs on the test set
# Test AUC is close to the "best" model AUC on the cross-validated training set which is a good indication of not suffering from overfitting
predicted_probabilities = grid_auc.predict_proba(X_test)
auc_score = roc_auc_score(y_test, predicted_probabilities[:, 1])
auc_score

In [ ]:
# Re-train the tuned model on the entire training data (not just on the 80% of it)
best_model_xgb = Pipeline([
    ("preprocessor", ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, NUM_VARS),
            ("cat", categorical_transformer, CAT_VARS)
            ]
    )),
    ("xgboost model", grid_auc.best_estimator_)
])


best_model_xgb.fit(X, y)
pred_xgb = pd.DataFrame(best_model_xgb.predict_proba(test_data), columns=["0", "1"])

# Creating data for submission
test_data_sub = pd.DataFrame(data={'ID':test_data['id'], 
                                   'PRED':pred_xgb["1"]})
test_data_sub

In [ ]:
# Exporting results
test_data_sub.to_csv('output/xgboost_pred_submission_v2.csv', header=True, index=False)